In [52]:
import urllib
import csv
import tweepy
import os
import tweepy
import csv
import os
import pandas as pd

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from string import punctuation
import collections
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import en_core_web_sm

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

# !pip3 install -U spacy
# !python3 -m spacy download en_core_web_sm

from sklearn.metrics import jaccard_score
from twitter import *
# Create variable
file_name = 'keys.py'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tnluser\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tnluser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [53]:
import pandas as pd
tweets = pd.read_csv("./Bewakoof_tweets-07Nov-01-emoji-Filtered.csv")
tweets.head(3)

,id,created_at,favorite_count,retweet_count,text,unicode_data
0,1.589230e+18,2022-11-06 12:29:37+00:00,26,6,Which one would you choose? 🤔\n\n#BewakoofAsks...,🤔
1,1.589200e+18,2022-11-06 10:30:04+00:00,1048,576,Our two favourite people in one frame 😍\nSo ho...,😍
2,1.589170e+18,2022-11-06 08:30:03+00:00,8,2,Nobody:\nLiterally nobody:\nThat one uncle in ...,NaN


In [54]:
with open("tweet_test.txt", "w", encoding="utf-8") as fp:
    fp.write("\n".join(tweets.text.to_list()))

In [55]:
tweets['scores'] = tweets['favorite_count'] + tweets['retweet_count']
top_tweets = tweets.sort_values(by = 'scores', ascending = False)
# top_tweets = top_tweets.head(500)
top_tweets.head()

,id,created_at,favorite_count,retweet_count,text,unicode_data,scores
614,1.534450e+18,2022-06-08 08:30:08+00:00,6530,1966,"""It's just a casual party. My girl gang &amp; ...",NaN,8496
626,1.534050e+18,2022-06-07 05:44:22+00:00,3928,1069,POV: You &amp; your girlfriend making a smashi...,😭𓃵,4997
488,1.544210e+18,2022-07-05 06:30:06+00:00,2715,861,Nobody:\nLiterally nobody:\nEvery girl in a ma...,NaN,3576
68,1.583380e+18,2022-10-21 09:00:58+00:00,2773,685,Pic 1: My hair on normal days\nPic 2: My hair ...,NaN,3458
601,1.535180e+18,2022-06-10 08:44:39+00:00,2245,1123,HBD BOSSBABE TEJASSWI ❤️\n\n#HappyBirthdayTeja...,❤️,3368


In [56]:
'''explore the time stamps and create groups'''
top_tweets['created_at'] = pd.to_datetime(top_tweets['created_at'])

In [57]:
top_tweets.head()

,id,created_at,favorite_count,retweet_count,text,unicode_data,scores
614,1.534450e+18,2022-06-08 08:30:08+00:00,6530,1966,"""It's just a casual party. My girl gang &amp; ...",NaN,8496
626,1.534050e+18,2022-06-07 05:44:22+00:00,3928,1069,POV: You &amp; your girlfriend making a smashi...,😭𓃵,4997
488,1.544210e+18,2022-07-05 06:30:06+00:00,2715,861,Nobody:\nLiterally nobody:\nEvery girl in a ma...,NaN,3576
68,1.583380e+18,2022-10-21 09:00:58+00:00,2773,685,Pic 1: My hair on normal days\nPic 2: My hair ...,NaN,3458
601,1.535180e+18,2022-06-10 08:44:39+00:00,2245,1123,HBD BOSSBABE TEJASSWI ❤️\n\n#HappyBirthdayTeja...,❤️,3368


In [58]:
# remove the hashtags, mentions and unwanted characters from the tweet texts
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df

clean_tweets = clean_text(tweets, 'text')
clean_tweets.head()

,id,created_at,favorite_count,retweet_count,text,unicode_data,scores
0,1.589230e+18,2022-11-06 12:29:37+00:00,26,6,which one would you choose bewakoofasks srk te...,🤔,32
1,1.589200e+18,2022-11-06 10:30:04+00:00,1048,576,our two favourite people in one frame so how h...,😍,1624
2,1.589170e+18,2022-11-06 08:30:03+00:00,8,2,nobodyliterally nobodythat one uncle in every ...,NaN,10
3,1.589140e+18,2022-11-06 06:30:03+00:00,14,3,newly launched flat knit sweaters feel fashio...,📢📢❄️💯,17
4,1.588890e+18,2022-11-05 13:30:21+00:00,32,3,people in delhi todaydelhi delhincr pollution,NaN,35


In [59]:
#remove stopwords, punctuations, lemmatize and tokenize word
# nlp = spacy.load("en_core_web_sm")   #working with english only, no support for swahili
nlp = en_core_web_sm.load()
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation) #already taken care of with the cleaning function.
stop.update(punctuation)
w_tokenizer = WhitespaceTokenizer()

            
def furnished(text):
    final_text = []
    for i in w_tokenizer.tokenize(text):
#     for i in text.split():
        if i.lower() not in stop:
            word = lemmatizer.lemmatize(i)
            final_text.append(word.lower())
    return " ".join(final_text)


            
tweets.text = tweets.text.apply(furnished)

C:\Users\Tnluser\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [60]:
tweets.sample(5)

,id,created_at,favorite_count,retweet_count,text,unicode_data,scores
320,1.560850e+18,2022-08-20 04:55:44+00:00,51,30,last day participate hurry answer win amazing ...,😍🤩🥳,81
240,1.567830e+18,2022-09-08 11:01:36+00:00,316,89,dono kidney aur aukat se bahar appleevent ios1...,🥲,405
475,1.545320e+18,2022-07-08 08:30:20+00:00,20,2,life seems good today life mekoffeewithkaran7 ...,😊,22
654,1.531630e+18,2022-05-31 13:42:45+00:00,10,0,cool tshirt sleek regional tamil graphic expre...,NaN,10
514,1.541680e+18,2022-06-28 06:56:39+00:00,139,105,contestalertdrop favorite amp relatable meme t...,🥳👇🏻,244


In [61]:
entertaining_content_words = '''recipe food cooking baking breakfast lunch dinner oven stove boil duck peach chocolate dessert ]
morning tea laugh joke funny weather shine My hair My face FalguniPathak NehaKakkar whatsapp sleepovers Rabir Deepika Amir Middle ClassFun
banter Joke gag joking kidding April fools hilarious cool whimsical haha hehe entertain laugh giggle Lol omg wtf l8r plz ttfl cheersguyswowarvoaussiegr8matem8
'''

In [62]:
relational_content_words = '''question ask you what do you think can you suggest suggestion ideas help congrats congratulations well done
    thanks thank you Quiz game test guess challenge Birthday Christmas Easter Boxing Day New Year Australia Day Good Friday
    Labor Day holiday public holiday Birthday Diwali Eid Independence day Holi Republic day Christmas Easter Boxing Day New Year
    Holiday Makar Sakranti Pongal Basant Panchami Maha Shivratri Mewar Baisakhi Bihu Thrissur Pooram Buddha Jayanti Eid Ul Fitr
    Rakshabandhan Janmashtmi Onam Ganesh Chaturthi Navratri Durga Puja Dussehra Diwali Gurupurab Melbourne Cup April fool Father’s Day
    Mother’s Day Monday Tuesday Wednesday Thursday Friday Saturday Sunday festive season Father's Day Mother's Day Monday Tuesday   Festive Season Festival diwali Wishingcomment if like if share if friends fans customers supporters
 '''

In [63]:
informational_content_words = '''
    Bewakoof
    Campus Sutra
    Snitch
    Dillinger
    Rigo
    Hubberholme
    Tistabene
    Bushirt
    Mad Over Print
    Difference Of Opinion
    Riag
    Adro
    Fugazee
    Breakbounce
    Almo
    Urban Scottish
    See Designs
    Craxystore
    Undrdawg By Bewakoof
    Old Grey
    Play By Bewakoof
    Alstyle
    Instafab Plus
    Style Quotient
    Conditions Apply
    Smugglerz
    Perfkt-U
    The Dry State
    Quarantine
    Cooft
    Fans Army
    Cinocci
    Try This
    Chkokko
    What's Down
    Door 74
    Tailor & Circus
    Blue Tyga
    Firangi Yarn
    Brown Mocha
    Redwolf
    Garcon
    Rozes.In
    Teeshut
    Fully Filmy
    Shvitz
    1947ind
    Jeans Cafe
    Price
    Rs
    http
    www
    .com
    Kurta
    Jeans
    Jacket
    Trunk
    Pyjama
    Track Pant
    Co-Ordinates
    Chino
    Trousers
    Sweater
    Shirt & Shorts Set
    Kurta Set
    Casual Pants
    Nightsuits
    Sweatshirt & Jogger Set
    T-Shirt & Pyjama Set
    Cloth Mask
    Lounge Pants
    Tracksuit
    Protective Outdoor Mask
    Hipster Trunk
    Lounge Shorts
    Nehru Jacket
    T-Shirt & Shorts Set
    Denim
    Blazer
    Hoodie And Shorts Set
    Pant
    Jumpsuit
    T-Shirt & Jogger Set
    range
    product
    phone
    email
    contact
    address
    location
    website
    get in touch
    reach
    connect
    did you know
    fact
    news
    Show
    win
    won
    award
    awarded
    received
    achieved
    successful
    medal
    trophy
    result
    review
    silver
    gold
    broze
    Solid
    Printed
    Graphic Print
    Typography
    Aop
    Color Block
    Floral Print
    Tie & Dye
    Striped
    Ethnic Motif
    Self Design
    Geometric Print
    Polka Print
    Washed
    Abstract
    Checked
    Embroidered
    Animal Print
    Camouflage
    Hyper Print
    Cut & Sew
    Woven
    Embellished
    Foil Print
    All Over Printed
    Chevron/Zig Zag
    Regular Fit
    Slim Fit
    Oversized
    Boyfriend Fit
    Relaxed Fit
    Loose Fit
    Skinny Fit
    Flared
    Straight Fit
    Wide Leg
    Super Loose
    Bootcut
    Carrot Fit
    Mom Fit
    Tapered Fit
    Unisex
    Sleeve
    Round Neck
    Hooded
    V-Neck
    Collar
    Spread Collar
    Mandarin Collar
    High Neck
    Notch Collar
    Lapel Collar
    Shoulder Straps
    Square Neck
    Scoop Neck
    Boat Neck
    Polo
    Halter Neck
    Henley Neck
    Sweetheart Neck
    Keyhole Neck
    Stand Collar
    Turtle Neck
    Off Shoulder
    Peter Pan
    Tie-Up
    Collarless
    One Shoulder
    Mock
    Strapless
    Hood
    T-Shirt
    Sweatshirt
    Kurta
    Hoodies
    Top
    Jeans
    Dress
    Joggers
    Kurta Set
    Shorts
    Top & Pyjama Set
    Pyjama
    Shirt & Pyjama Set
    Boxer
    Co-Ordinates
    Shrug
    Top & Shorts Set
    Varsity
    Jacket
    Nightsuits
    Shirt & Shorts Set
    Sweater
    Tank Top
    Wind Cheater
    Shirt
    Short Top
    Trackpants
    Lounge Shorts
    Regular Shorts
    Tights
    Blouse
    T-Shirt & Pyjama Set
    Pants
    Top & Capri Set
    Jumpsuit
    Leggings
    Lounge Pants
    T-Shirt Dress
    Everyday
    Vest
    Bra
    Super Skinny
    T-Shirt & Jogger Set
    T-Shirt & Shorts Set
    Boyfriend
    Crop
    Flared
    Short
    Cargo Jogger
 
'''

In [64]:
reminurative_content_words = '''
    Special
    discount
    exclusive
    deal
    sale
    promotion
    clearance
    bargain
    on sale
    marked down
    free
    gift
    Win
    reward
    free
    prize
    win
    reward
    free
    prize
'''

In [65]:
nlp = en_core_web_sm.load()
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)
w_tokenizer = WhitespaceTokenizer()

# clean the set of words
            
def furnished(text):
    final_text = []
    for i in text.split():
        if i.lower() not in stop:
            word = lemmatizer.lemmatize(i)
            final_text.append(word.lower())
    return " ".join(final_text)

C:\Users\Tnluser\AppData\Roaming\Python\Python39\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [66]:
entertaining = furnished(entertaining_content_words)
informational = furnished(informational_content_words)
relational = furnished(relational_content_words)
reminurative = furnished(reminurative_content_words)

In [67]:
# delete duplicates
string1 = entertaining
words = string1.split()
entertaining = " ".join(sorted(set(words), key=words.index))
entertaining

'recipe food cooking baking breakfast lunch dinner oven stove boil duck peach chocolate dessert morning tea laugh joke funny weather shine hair face falgunipathak nehakakkar whatsapp sleepover rabir deepika amir middle classfun banter gag joking kidding april fool hilarious cool whimsical haha hehe entertain giggle lol omg wtf l8r plz ttfl cheersguyswowarvoaussiegr8matem8'

In [68]:
# delete duplicates
string1 = informational
words = string1.split()
informational = " ".join(sorted(set(words), key=words.index))
informational

"bewakoof campus sutra snitch dillinger rigo hubberholme tistabene bushirt mad print difference opinion riag adro fugazee breakbounce almo urban scottish see designs craxystore undrdawg old grey play alstyle instafab plus style quotient conditions apply smugglerz perfkt-u dry state quarantine cooft fans army cinocci try chkokko what's door 74 tailor circus blue tyga firangi yarn brown mocha redwolf garcon rozes.in teeshut fully filmy shvitz 1947ind jeans cafe price rs http www .com kurta jacket trunk pyjama track pant co-ordinates chino trousers sweater shirt shorts set casual pants nightsuits sweatshirt jogger t-shirt cloth mask lounge tracksuit protective outdoor hipster nehru denim blazer hoodie jumpsuit range product phone email contact address location website get touch reach connect know fact news show win award awarded received achieved successful medal trophy result review silver gold broze solid printed graphic typography aop color block floral tie dye striped ethnic motif sel

In [69]:
# delete duplicates
string1 = relational
words = string1.split()
relational = " ".join(sorted(set(words), key=words.index))
relational

"question ask think suggest suggestion idea help congrats congratulation well done thanks thank quiz game test guess challenge birthday christmas easter boxing day new year australia good friday labor holiday public diwali eid independence holi republic makar sakranti pongal basant panchami maha shivratri mewar baisakhi bihu thrissur pooram buddha jayanti ul fitr rakshabandhan janmashtmi onam ganesh chaturthi navratri durga puja dussehra gurupurab melbourne cup april fool father’s mother’s monday tuesday wednesday thursday saturday sunday festive season father's mother's festival wishingcomment like share friend fan customer supporter"

In [70]:
# delete duplicates
string1 = reminurative
words = string1.split()
reminurative = " ".join(sorted(set(words), key=words.index))
reminurative

'special discount exclusive deal sale promotion clearance bargain marked free gift win reward prize'

In [71]:
'''Vectorizing the sets of words, then standardizing them. TFIDF will be used in order to take care of the least 
frequent words. Standardizing is cause TFIDF favors long sentences and there'll be inconsistencies between the length 
of the tweets and the length of set of words.'''


def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = TfidfVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

In [72]:
entertaining_vector = get_vectors(entertaining)
informational_vector = get_vectors(informational)
relational_vector = get_vectors(relational)
reminurative_vector = get_vectors(reminurative)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass input=['recipe food cooking baking breakfast lunch dinner oven stove boil duck peach chocolate dessert morning tea laugh joke funny weather shine hair face falgunipathak nehakakkar whatsapp sleepover rabir deepika amir middle classfun banter gag joking kidding april fool hilarious cool whimsical haha hehe entertain giggle lol omg wtf l8r plz ttfl cheersguyswowarvoaussiegr8matem8'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass input=["bewakoof campus sutra snitch dillinger rigo hubberholme tistabene bushirt mad print difference opinion riag adro fugazee breakbounce almo urban scottish see designs craxystore undrdawg old grey play alstyle instafab plus style quotient c

In [73]:
## Vectorizing the tweets
tv=TfidfVectorizer()
# tweets_bowl = tweets_bowl.tweets.apply(get_vectors)
# tweets_bowl.head()
tfidf_tweets =tv.fit_transform(tweets.text)

In [74]:
'''Jaccard similarity is good for cases where duplication does not matter, 
cosine similarity is good for cases where duplication matters while analyzing text similarity. For two product descriptions, 
it will be better to use Jaccard similarity as repetition of a word does not reduce their similarity.'''

def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)
# jaccard_score(socialvector, economic_vector)

#for similarity of 1 and 2 of column1
# jaccard_similarity('dog lion a dog','dog is cat')


def get_scores(group,tweets):
    scores = []
    for tweet in tweets:
        s = jaccard_similarity(group, tweet)
        scores.append(s)
    return scores

In [75]:
# informational scores
i_scores = get_scores(informational, tweets.text.to_list())
i_scores[-10:]

[0.4857142857142857,
 0.6285714285714286,
 0.6,
 0.22857142857142856,
 0.45714285714285713,
 0.6111111111111112,
 0.4864864864864865,
 0.4864864864864865,
 0.40540540540540543,
 0.7027027027027027]

In [76]:
# entertaining scores
e_scores = get_scores(entertaining, tweets.text.to_list())
e_scores[-10:]

[0.5925925925925926,
 0.7777777777777778,
 0.8076923076923077,
 0.3076923076923077,
 0.6153846153846154,
 0.75,
 0.6428571428571429,
 0.6428571428571429,
 0.5357142857142857,
 0.6875]

In [77]:
# relational scores
rel_scores = get_scores(relational, tweets.text.to_list())
rel_scores[-10:]

[0.5862068965517241,
 0.7586206896551724,
 0.7241379310344828,
 0.27586206896551724,
 0.5517241379310345,
 0.6774193548387096,
 0.5806451612903226,
 0.5806451612903226,
 0.4838709677419355,
 0.6285714285714286]

In [78]:
# reminurative scores
remin_scores = get_scores(reminurative, tweets.text.to_list())
remin_scores[-10:]

[0.6666666666666666,
 0.8,
 0.76,
 0.2916666666666667,
 0.56,
 0.7037037037037037,
 0.5925925925925926,
 0.6538461538461539,
 0.5384615384615384,
 0.59375]

In [79]:
'''new df with names, and the jaccard scores for each group'''

data  = {'tweets':tweets.text.to_list(), 'enterainment_score':e_scores,
         'informational_score': i_scores, 'relational_score':rel_scores, 'reminurative_scores':remin_scores}
scores_df = pd.DataFrame(data)
scores_df.head(20)

,tweets,enterainment_score,informational_score,relational_score,reminurative_scores
0,one would choose bewakoofasks srk teamindia,0.730769,0.542857,0.655172,0.750000
1,two favourite people one frame high joshshehna...,0.884615,0.657143,0.793103,0.769231
2,nobodyliterally nobodythat one uncle every wed...,0.807692,0.600000,0.724138,0.760000
3,newly launched flat knit sweater feel fashiona...,0.846154,0.628571,0.758621,0.800000
4,people delhi todaydelhi delhincr pollution,0.576923,0.428571,0.517241,0.520000
5,cricketer haswhat hashappybirthdayviratkohli t...,0.678571,0.513514,0.612903,0.629630
6,celebrate king viratkohlis birthday contest pa...,0.766667,0.631579,0.696970,0.724138
7,arrived new flat knit sweatersseasons stylish ...,0.884615,0.657143,0.793103,0.840000
8,shark tank india s2 playing heart sharktankind...,0.703704,0.527778,0.633333,0.592593
9,like friend salmankhan bollywoodmemes,0.653846,0.485714,0.586207,0.600000


In [80]:
'''Actual assigning of classes to the tweets'''

def get_clusters(l1, l2, l3, l4):
    enter = []
    infor = []
    relat = []
    remin = []
    for i, j, k, l in zip(l1, l2, l3, l4):
        m = max(i, j, k, l)
        if m == i:
            enter.append(1)
        else:
            enter.append(0)
        if m == j:
            infor.append(1)
        else:
            infor.append(0)        
        if m == k:
            relat.append(1)
        else:
            relat.append(0)  
        if m == l:
            remin.append(1)
        else:
            remin.append(0)   
            
    return enter, infor, relat, remin

In [83]:
l1 = scores_df.enterainment_score.to_list()
l2 = scores_df.informational_score.to_list()
l3 = scores_df.relational_score.to_list()
l4 = scores_df.reminurative_scores.to_list()

enter, infor, relat, remin = get_clusters(l1, l2, l3, l4)

In [84]:
data = {'tweets': scores_df.tweets.to_list(), 'enterainment':enter, 'informational':infor, 'relational':relat, 'reminurative': remin}
cluster_df = pd.DataFrame(data)
cluster_df.head(5)
cluster_df.to_csv('.\Bewakoof_tweets-07Nov-01-emoji-Filtered_Classified.csv') 